In [ ]:
root_git = ''
root_wiki = ''

# Imports

In [63]:
import pandas as pd
from Code.data_visualization import *
from tqdm import tqdm

def drop_from_interval(df: pd.DataFrame, min_interval: float, max_interval: float, n_samples_to_drop: int) -> pd.DataFrame:
    tmp = df[df['a%'] >= min_interval]
    if max_interval == 1:
        tmp = tmp[tmp['a%'] <= max_interval]
    else:
        tmp = tmp[tmp['a%'] < max_interval]
    to_drop_rows = tmp.sample(n_samples_to_drop)
    to_drop = list(to_drop_rows.index)

    return df.drop(to_drop), to_drop_rows

def augment_interval(df_old: pd.DataFrame, df_for_augmentation: pd.DataFrame, min_interval: float, max_interval: float, n_samples_to_add: int) -> pd.DataFrame:
    tmp = df_for_augmentation[df_for_augmentation['a%'] >= min_interval]
    if max_interval == 1:
        tmp = tmp[tmp['a%'] <= max_interval]
    else:
        tmp = tmp[tmp['a%'] < max_interval]
    tmp = tmp.sample(n_samples_to_add)
    result = pd.concat([df_old, tmp], axis=0)
    return result

def get_table_set(df: pd.DataFrame) -> set:
    out = []
    for r in tqdm(range(df.shape[0])):
        out.append(df.iloc[r]['r_id'])
        out.append(df.iloc[r]['s_id'])
    return set(out)

def check_no_repetitions(s1: set, s2: set) -> bool:
    for t in tqdm(s1):
        if t in s2:
            return True
    return False

def get_couple_set(df: pd.DataFrame) -> set:
    out = []
    for r in range(df.shape[0]):
        tmp1 = (df.iloc[r]['r_id'], df.iloc[r]['s_id'])
        tmp2 = (df.iloc[r]['s_id'], df.iloc[r]['r_id'])
        out.append(tmp1)
        out.append(tmp2)
    return set(out)

def drop_duplicate_samples(pivot: pd.DataFrame, target: pd.DataFrame) -> pd.DataFrame:
    print('Generating pivot couple set')
    pivot_couple_set = get_couple_set(pivot)

    print('Generating target couple set')
    target_couple_set = get_couple_set(target)

    samples_to_drop = []

    print('Generating Samples to drop list')
    for t in target_couple_set:
        if t in pivot_couple_set:
            samples_to_drop.append(t)
    
    samples_to_drop = set(samples_to_drop)

    print('Generating rows to drop list')
    rows_to_drop = []
    for r in range(target.shape[0]):
        tmp1 = (target.iloc[0]['r_id'], target.iloc[0]['s_id'])
        tmp2 = (target.iloc[0]['s_id'], target.iloc[0]['r_id'])
        if (tmp1 in samples_to_drop) or (tmp2 in samples_to_drop):
            rows_to_drop.append(r)

    print('Dropping rows')
    out = target.drop(rows_to_drop)

    return out

def remove_exact_matches(df: pd.DataFrame):
    out = df[df['a%']!=1]
    return out

Preliminar

Fixing tuples for data augmentation

Train

In [4]:
t1 = pd.read_csv(root_git+'/balanced_datasets/lsh_candidates/train_stats_200000.csv')
t2 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/train_stats_400000.csv')
t3 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/train_stats_600000.csv')
t4 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/train_stats_800000.csv')
t5 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/train_stats_1000000.csv')

In [5]:
result = pd.concat([t1, t2, t3, t4, t5], axis=0)

In [6]:
result= result.drop('id', axis=1)
result= result.drop('total_time', axis=1)

In [ ]:
result.head(5)

In [8]:
result.to_csv(root_git+'balanced_datasets/unbalanced_datasets/train_unbalanced.csv', index=False)

In [9]:
pd.read_csv(root_git+'balanced_datasets/lsh_candidates/test_stats_200000.csv').drop('id',axis=1).to_csv(root_git+'balanced_datasets/unbalanced_datasets/test_unbalanced.csv', index=False)

In [10]:
pd.read_csv(root_git+'balanced_datasets/lsh_candidates/valid_stats_200000.csv').drop('id',axis=1).to_csv(root_git+'balanced_datasets/unbalanced_datasets/valid_unbalanced.csv', index=False)

Test

In [11]:
te1 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/test_stats_200000.csv')
te2 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/test_stats_300000.csv')
te3 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/test_stats_500000.csv')
te4 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/test_stats_600000.csv')

tetot = result = pd.concat([te1, te2, te3, te4], axis=0)

result_te= tetot.drop('id', axis=1)
result_te['a%'] = result_te['a%'].fillna(0)

result_te.to_csv(root_git+'balanced_datasets/unbalanced_datasets/test_unbalanced.csv', index=False)

In [12]:
ttttest = pd.read_csv(root_git+'balanced_datasets/unbalanced_datasets/test_unbalanced.csv')

In [ ]:
ttttest

Valid

In [14]:
v1 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/valid_stats_200000.csv')
v2 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/valid_stats_300000.csv')
v3 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/valid_stats_500000.csv')
v4 = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/valid_stats_600000.csv')

vtot = result = pd.concat([v1, v2, v3, v4], axis=0)

result_v= vtot.drop('id', axis=1)
result_v['a%'] = result_v['a%'].fillna(0)

result_v.to_csv(root_git+'balanced_datasets/unbalanced_datasets/valid_unbalanced.csv', index=False)

In [ ]:
vvvvalid = pd.read_csv(root_git+'balanced_datasets/unbalanced_datasets/valid_unbalanced.csv')
vvvvalid

???????????????????????

In [16]:
test_unbalanced = pd.read_csv(root_git+'balanced_datasets/unbalanced_datasets/test_unbalanced.csv')
test_unbalanced = test_unbalanced.drop('total_time', axis=1)
test_unbalanced['a%'] = test_unbalanced['a%'].fillna(0)

test_data_augm = pd.read_csv(root_git+'balanced_datasets/lsh_candidates/test_stats_300000.csv')
test_data_augm = test_data_augm.drop('total_time', axis=1)
test_data_augm = test_data_augm.drop('id', axis=1)

# Gittables

In [21]:
def drop_from_interval(df: pd.DataFrame, min_interval: float, max_interval: float, n_samples_to_drop: int) -> pd.DataFrame:
    tmp = df[df['a%'] >= min_interval]
    if max_interval == 1:
        tmp = tmp[tmp['a%'] <= max_interval]
    else:
        tmp = tmp[tmp['a%'] < max_interval]
    to_drop_rows = tmp.sample(n_samples_to_drop)
    to_drop = list(to_drop_rows.index)

    return df.drop(to_drop), to_drop_rows

def augment_interval(df_old: pd.DataFrame, df_for_augmentation: pd.DataFrame, min_interval: float, max_interval: float, n_samples_to_add: int) -> pd.DataFrame:
    tmp = df_for_augmentation[df_for_augmentation['a%'] >= min_interval]
    if max_interval == 1:
        tmp = tmp[tmp['a%'] <= max_interval]
    else:
        tmp = tmp[tmp['a%'] < max_interval]
    tmp = tmp.sample(n_samples_to_add)
    result = pd.concat([df_old, tmp], axis=0)
    return result

def get_table_set(df: pd.DataFrame) -> set:
    out = []
    for r in tqdm(range(df.shape[0])):
        out.append(df.iloc[r]['r_id'])
        out.append(df.iloc[r]['s_id'])
    return set(out)

def check_no_repetitions(s1: set, s2: set) -> bool:
    for t in tqdm(s1):
        if t in s2:
            return True
    return False

def get_couple_set(df: pd.DataFrame) -> set:
    out = []
    for r in range(df.shape[0]):
        tmp1 = (df.iloc[r]['r_id'], df.iloc[r]['s_id'])
        tmp2 = (df.iloc[r]['s_id'], df.iloc[r]['r_id'])
        out.append(tmp1)
        out.append(tmp2)
    return set(out)

def drop_duplicate_samples(pivot: pd.DataFrame, target: pd.DataFrame) -> pd.DataFrame:
    print('Generating pivot couple set')
    pivot_couple_set = get_couple_set(pivot)

    print('Generating target couple set')
    target_couple_set = get_couple_set(target)

    samples_to_drop = []

    print('Generating Samples to drop list')
    for t in target_couple_set:
        if t in pivot_couple_set:
            samples_to_drop.append(t)
    
    samples_to_drop = set(samples_to_drop)

    print('Generating rows to drop list')
    rows_to_drop = []
    for r in range(target.shape[0]):
        tmp1 = (target.iloc[0]['r_id'], target.iloc[0]['s_id'])
        tmp2 = (target.iloc[0]['s_id'], target.iloc[0]['r_id'])
        if (tmp1 in samples_to_drop) or (tmp2 in samples_to_drop):
            rows_to_drop.append(r)

    print('Dropping rows')
    out = target.drop(rows_to_drop)

    return out


## Data Augmentation Data Generation

In [23]:
train_tables = pd.read_csv(root_git+'balanced_datasets/unbalanced_datasets/train_unbalanced_not_augmented.csv')
test_tables = pd.read_csv(root_git+'balanced_datasets/unbalanced_datasets/test_unbalanced_not_augmented.csv')
valid_tables = pd.read_csv(root_git+'balanced_datasets/unbalanced_datasets/valid_unbalanced_not_augmented.csv')

In [ ]:
train_tables_set = get_table_set(train_tables)
test_tables_set = get_table_set(test_tables)
valid_tables_set = get_table_set(valid_tables)

In [ ]:
check_no_repetitions(train_tables_set, test_tables_set)

In [ ]:
check_no_repetitions(train_tables_set, valid_tables_set)

In [ ]:
check_no_repetitions(valid_tables_set, test_tables_set)

In [28]:
with open(root_git+'balanced_datasets/data_augementation_samples/train_tables_set.pkl', 'wb') as f:
    pickle.dump(train_tables_set,f)

with open(root_git+'balanced_datasets/data_augementation_samples/test_tables_set.pkl', 'wb') as f:
    pickle.dump(test_tables_set,f)

with open(root_git+'balanced_datasets/data_augementation_samples/valid_tables_set.pkl', 'wb') as f:
    pickle.dump(valid_tables_set,f)

In [29]:
with open(root_git+'balanced_datasets/data_augementation_samples/train_tables_set.pkl', 'rb') as f:
    train_tables_set = pickle.load(f)

with open(root_git+'balanced_datasets/data_augementation_samples/test_tables_set.pkl', 'rb') as f:
    test_tables_set = pickle.load(f)

with open(root_git+'balanced_datasets/data_augementation_samples/valid_tables_set.pkl', 'rb') as f:
    valid_tables_set = pickle.load(f)

## Train

In [43]:
train_unbalanced = pd.read_csv(root_git+'balanced_datasets/unbalanced_datasets/train_unbalanced.csv')
#train_unbalanced = train_unbalanced.drop('total_time', axis=1)
train_unbalanced['a%'] = train_unbalanced['a%'].fillna(0)

In [ ]:
train_unbalanced

In [ ]:
plot_data_distribution(train_unbalanced, 'a%')

Cutting columns that are too high

In [46]:
train_2 = drop_from_interval(train_unbalanced, 0.5, 0.6, 363_117)[0]

In [47]:
train_3 = drop_from_interval(train_2, 0.6, 0.7, 20_470)[0]

In [48]:
train_4, dropped_78 = drop_from_interval(train_3, 0.7, 0.8, 18_044) ##

In [49]:
train_5, dropped_79 = drop_from_interval(train_4, 0.8, 0.9, 3_821) ##

In [ ]:
train_6 = drop_from_interval(train_5, 0.9, 1, 186_098)[0]
plot_data_distribution(train_6, 'a%')

In [51]:
train_6.to_csv(root_git+'balanced_datasets/unbalanced_datasets/train_unbalanced_not_augmented.csv', index=False)

Data augmentation

In [69]:
augm_train_1 = pd.read_csv(root_git+'balanced_datasets/data_augementation_samples/train_stats_1000000.csv')
augm_train_2 = pd.read_csv(root_git+'balanced_datasets/data_augementation_samples/train_stats_2000000.csv')
augm_train = pd.concat([augm_train_1, augm_train_2], axis=0)
augm_train = augm_train.drop('total_time', axis=1)
augm_train = augm_train.drop('id', axis=1)
augm_train['a%'] = augm_train['a%'].fillna(0)
augm_train = drop_duplicate_samples(train_6, augm_train)
augm_train.to_csv(root_git+'balanced_datasets/data_augementation_samples/train_stats_concat.csv', index=False)
augm_train = pd.read_csv(root_git+'balanced_datasets/data_augementation_samples/train_stats_concat.csv')

In [ ]:
plot_data_distribution(augm_train, 'a%')

In [87]:
train_augmented = augment_interval(train_6, augm_train, 0,0.1,2_959)
train_augmented = augment_interval(train_augmented, augm_train, 0.1, 0.2, 17_619)
train_augmented = augment_interval(train_augmented, augm_train, 0.2, 0.3, 26_631)
train_augmented = augment_interval(train_augmented, augm_train, 0.3, 0.4, 36_845)
train_augmented = augment_interval(train_augmented, augm_train, 0.4, 0.5, 7_496)

In [ ]:
plot_data_distribution(train_augmented, 'a%')

In [ ]:
train_augmented.to_csv(root_git+'balanced_datasets/train.csv', index=False)

## Fixing broken triples:

In [6]:
train_data_filtered = pd.read_csv(root_git+'armadillo_corrections/train_filtered.csv')

In [ ]:
train_data_filtered

In [ ]:
plot_data_distribution(train_data_filtered)

In [9]:
train_data_for_augmentation = pd.read_csv(root_git+'armadillo_corrections/train_stats_data_augmentation_seeds.csv')

In [ ]:
train_data_for_augmentation.shape

In [ ]:
plot_data_distribution(train_data_for_augmentation)

In [ ]:
train_data_for_augmentation

In [ ]:
train_data_for_augmentation = drop_duplicate_samples(train_data_filtered, train_data_for_augmentation)

In [ ]:
train_data_for_augmentation.shape

In [14]:
train_data_for_augmentation = train_data_for_augmentation[train_data_for_augmentation['seeds']==0]

In [ ]:
train_data_for_augmentation

In [ ]:
pd.isna(train_data_for_augmentation['a%']).sum()

In [ ]:
train_data_for_augmentation = train_data_for_augmentation[pd.isna(train_data_for_augmentation['a%'])]
train_data_for_augmentation['a%'] = train_data_for_augmentation['a%'].fillna(0)
train_data_for_augmentation.shape

In [ ]:
train_data_for_augmentation['seeds'].sum()

In [31]:
train_data_for_augmentation = train_data_for_augmentation.drop(['id', 'seeds'], axis=1)

In [32]:
train_data_augmented = augment_interval(train_data_filtered, train_data_for_augmentation, 0, 0.1, 2_657)

In [ ]:
plot_data_distribution(train_data_augmented)

In [ ]:
pd.isna(train_data_augmented['a%']).sum()

In [36]:
train_data_augmented.to_csv(root_git+'armadillo_corrections/train_filtered_augmented.csv', index=False)

## Building training dataset without perfect matches

In [3]:
df_with_ones = pd.read_csv(root_git+'armadillo_corrections/train_filtered_augmented.csv')

In [4]:
df_no_ones = remove_exact_matches(df_with_ones)

In [ ]:
plot_data_distribution(df_no_ones)

In [ ]:
df_no_ones

In [12]:
data_augm = pd.read_csv(root_git+'armadillo_corrections/train_stats_banding_seeds.csv')

In [ ]:
plot_data_distribution(data_augm)

In [14]:
data_augm = remove_exact_matches(data_augm)

In [16]:
data_augm = data_augm[data_augm['a%']>=0.9]

In [ ]:
plot_data_distribution(data_augm)

In [ ]:
data_augm = drop_duplicate_samples(df_no_ones, data_augm)

In [ ]:
data_augm.describe()

In [31]:
data_augm = data_augm.drop(columns=['id', 'seeds','total_time'])

In [32]:
df_no_ones_balanced = augment_interval(df_no_ones, data_augm, 0.9, 1, 36_368)

In [ ]:
plot_data_distribution(df_no_ones_balanced)

In [36]:
df_no_ones_balanced.to_csv(root_git+'train_no_perfect_matches.csv', index=False)

## Test

In [4]:
test_unbalanced = pd.read_csv(root_git+'balanced_datasets/unbalanced_datasets/test_unbalanced.csv')
test_unbalanced = test_unbalanced.drop('total_time', axis=1)
test_unbalanced['a%'] = test_unbalanced['a%'].fillna(0)

In [ ]:
plot_data_distribution(test_unbalanced, 'a%')

In [10]:
test_2 = drop_from_interval(test_unbalanced, 0, 0.1, 21_590)[0]
test_2 = drop_from_interval(test_2, 0.1, 0.2, 7_443)[0]
test_2 = drop_from_interval(test_2, 0.2, 0.3, 2_937)[0]
test_2 = drop_from_interval(test_2, 0.4, 0.5, 11_163)[0]
test_3 = drop_from_interval(test_2, 0.5, 0.6, 228_664)[0]
test_4 = drop_from_interval(test_3, 0.6, 0.7, 23_636)[0]
test_5 = drop_from_interval(test_4, 0.7, 0.8, 20_561)[0]
test_6 = drop_from_interval(test_5, 0.8, 0.9, 14_011)[0]
test_6 = drop_from_interval(test_6, 0.9, 1, 103_225)[0]

In [ ]:
plot_data_distribution(test_6, 'a%')

Data augmentation

In [ ]:
augm_test= pd.read_csv(root_git+'balanced_datasets/data_augementation_samples/test_stats_500000.csv')
augm_test = augm_test.drop('total_time', axis=1)
augm_test = augm_test.drop('id', axis=1)
augm_test['a%'] = augm_test['a%'].fillna(0)
augm_test = drop_duplicate_samples(test_6, augm_test)
augm_test.to_csv(root_git+'balanced_datasets/data_augementation_samples/test_stats_cleaned.csv', index=False)
augm_test = pd.read_csv(root_git+'balanced_datasets/data_augementation_samples/test_stats_cleaned.csv')

In [ ]:
plot_data_distribution(augm_test, 'a%')

In [17]:
test_augmented = augment_interval(test_6, augm_test, 0.3,0.4, 2_790)

In [ ]:
plot_data_distribution(test_augmented, 'a%')

In [19]:
test_augmented.to_csv(root_git+'balanced_datasets/test.csv', index=False)

In [ ]:
tg = pd.read_csv(root_git+'balanced_datasets/test.csv')
data_augm = pd.read_csv(root_git+'balanced_datasets/data_augementation_samples/test_stats_500000.csv')
data_augm['a%'] = data_augm['a%'].fillna(0)
d_0 = data_augm[data_augm['a%']==0]
t_check = d_0.iloc[-2][1:4]
for r in range(tg.shape[0]):
    if ((tg.iloc[r]['r_id']==t_check['r_id']) and (tg.iloc[r]['s_id']==t_check['s_id'])) or ((tg.iloc[r]['r_id']==t_check['s_id']) and (tg.iloc[r]['s_id']==t_check['r_id'])):
        print('nope')
        break
print('ok')
tg.iloc[92616] = d_0.iloc[-2][1:4]
tg.to_csv(root_git+'balanced_datasets/test.csv', index=False)

## Fixing broken triples

In [ ]:
test_data_filtered = pd.read_csv(root_git+'armadillo_corrections/test_filtered.csv')
test_data_filtered

In [ ]:
plot_data_distribution(test_data_filtered)

In [ ]:
test_data_for_augmentation = pd.read_csv(root_git+'armadillo_corrections/test_stats_data_augmentation_seeds.csv')
plot_data_distribution(test_data_for_augmentation)

In [ ]:
test_data_for_augmentation = drop_duplicate_samples(test_data_filtered, test_data_for_augmentation)

In [ ]:
test_data_for_augmentation = test_data_for_augmentation[test_data_for_augmentation['seeds']==0]
test_data_for_augmentation = test_data_for_augmentation[pd.isna(test_data_for_augmentation['a%'])]
test_data_for_augmentation['a%'] = test_data_for_augmentation['a%'].fillna(0)
test_data_for_augmentation.shape

In [21]:
test_data_for_augmentation = test_data_for_augmentation.drop(['id', 'seeds'], axis=1)

In [22]:
test_data_augmented = augment_interval(test_data_filtered, test_data_for_augmentation, 0, 0.1, 430)

In [ ]:
plot_data_distribution(test_data_augmented)

In [ ]:
test_data_augmented.iloc[99591]

In [25]:
test_data_augmented.to_csv(root_git+'armadillo_corrections/test_filtered_augmented.csv', index=False)

## Validation

In [22]:
valid_unbalanced = pd.read_csv(root_git+'balanced_datasets/unbalanced_datasets/valid_unbalanced.csv')
valid_unbalanced = valid_unbalanced.drop('total_time', axis=1)
valid_unbalanced['a%'] = valid_unbalanced['a%'].fillna(0)

In [ ]:
plot_data_distribution(valid_unbalanced, 'a%')

In [24]:
valid_2 = drop_from_interval(valid_unbalanced, 0, 0.1, 19_770)[0]
valid_2 = drop_from_interval(valid_2, 0.1, 0.2, 9_556)[0]
valid_2 = drop_from_interval(valid_2, 0.2, 0.3, 2_483)[0]
valid_2 = drop_from_interval(valid_2, 0.4, 0.5, 12_259)[0]
valid_3 = drop_from_interval(valid_2, 0.5, 0.6, 230_147)[0]
valid_4 = drop_from_interval(valid_3, 0.6, 0.7, 22_245)[0]
valid_5 = drop_from_interval(valid_4, 0.7, 0.8, 18_914)[0]
valid_5 = drop_from_interval(valid_5, 0.8, 0.9, 15_190)[0]
valid_6 = drop_from_interval(valid_5, 0.9, 1, 102_066)[0]

In [ ]:
plot_data_distribution(valid_6, 'a%')

Data augmentation

In [ ]:
augm_valid = pd.read_csv(root_git+'balanced_datasets/data_augementation_samples/valid_stats_500000.csv')
augm_valid = augm_valid.drop('total_time', axis=1)
augm_valid = augm_valid.drop('id', axis=1)
augm_valid['a%'] = augm_valid['a%'].fillna(0)
augm_valid = drop_duplicate_samples(valid_6, augm_valid)
augm_valid.to_csv(root_git+'balanced_datasets/data_augementation_samples/valid_stats_cleaned.csv', index=False)
augm_valid = pd.read_csv(root_git+'balanced_datasets/data_augementation_samples/valid_stats_cleaned.csv')

In [ ]:
plot_data_distribution(augm_valid, 'a%')

In [28]:
valid_augmented = augment_interval(valid_6, augm_valid, 0.3,0.4, 2_811)

In [ ]:
plot_data_distribution(valid_augmented, 'a%')

In [30]:
valid_augmented.to_csv(root_git+'balanced_datasets/valid.csv', index=False)

## Fixing broken triples

In [ ]:
valid_data_filtered = pd.read_csv(root_git+'armadillo_corrections/valid_filtered.csv')
valid_data_for_augmentation = pd.read_csv(root_git+'armadillo_corrections/valid_stats_data_augmentation_seeds.csv')
valid_data_for_augmentation = drop_duplicate_samples(valid_data_filtered, valid_data_for_augmentation)

In [3]:
valid_data_for_augmentation = valid_data_for_augmentation[valid_data_for_augmentation['seeds']==0]
valid_data_for_augmentation = valid_data_for_augmentation[pd.isna(valid_data_for_augmentation['a%'])]
valid_data_for_augmentation['a%'] = valid_data_for_augmentation['a%'].fillna(0)
valid_data_for_augmentation.shape
valid_data_for_augmentation = valid_data_for_augmentation.drop(['id', 'seeds'], axis=1)

In [ ]:
plot_data_distribution(valid_data_filtered)

In [6]:
valid_data_augmented = augment_interval(valid_data_filtered, valid_data_for_augmentation, 0, 0.1, 377)

In [ ]:
plot_data_distribution(valid_data_augmented)

In [8]:
valid_data_augmented.to_csv(root_git+'armadillo_corrections/valid_filtered_augmented.csv', index=False)

# Results Validation

In [10]:
train = pd.read_csv(root_git+'armadillo_corrections/train_filtered_augmented.csv')
test = pd.read_csv(root_git+'armadillo_corrections/test_filtered_augmented.csv')
valid = pd.read_csv(root_git+'armadillo_corrections/valid_filtered_augmented.csv')

In [ ]:
plot_data_distribution(test)

In [ ]:
test.head(5)

In [ ]:
valid.head(5)

# Stats Computation

In [19]:
import numbers

def compute_counts(t: pd.DataFrame) -> tuple:
    value_count = {}
    n_nans = 0
    n_str = 0
    n_num = 0
    n_bools = 0

    for r in range(t.shape[0]):
        for c in range(t.shape[1]):
            v = t.iloc[r][c]
            value_count[v] = 0
            if pd.isna(v):
                n_nans += 1
            elif isinstance(v, bool):
                n_bools += 1
            elif isinstance(v, numbers.Number):
                n_num += 1
            else:
                n_str += 1
    return len(value_count.keys()), n_num, n_str, n_nans, n_bools

def compute_table_stats(table_dict: dict | str, outpath: str) -> None:
    if isinstance(table_dict, str):
        with open(table_dict, 'rb') as f:
            table_dict = pickle.load(f)
    new_cols = {
        'table_name':[],
        'n_cols':[],
        'n_rows':[],
        'area':[],
        'n_distinct_values':[],
        'n_numerical':[],
        'n_textual':[],
        'n_nans':[],
        'n_bools':[]
    }

    for k in tqdm(table_dict.keys()):
        t = table_dict[k]
        rows = t.shape[0]
        cols = t.shape[1]
        area = rows * cols
        n_distinct_values, n_numerical, n_textual, n_nans, n_bools = compute_counts(t)
        new_cols['table_name'].append(k)
        new_cols['n_cols'].append(cols)
        new_cols['n_rows'].append(rows)
        new_cols['area'].append(area)
        new_cols['n_distinct_values'].append(n_distinct_values)
        new_cols['n_numerical'].append(n_numerical)
        new_cols['n_textual'].append(n_textual)
        new_cols['n_nans'].append(n_nans)
        new_cols['n_bools'].append(n_bools)
    
    out = pd.DataFrame(new_cols)
    out.to_csv(outpath)
    print(out.describe())


In [27]:
with open(root_git+'table_dict_796970_good.pkl','rb') as f:
    table_dict = pickle.load(f)

In [ ]:
compute_stats(table_dict, root_git+'gittables_stats.csv')

# Tests

In [ ]:
# ag_train = get_table_set(pd.read_csv(root_git+'balanced_datasets/train.csv'))
# ag_test = get_table_set(pd.read_csv(root_git+'balanced_datasets/test.csv'))
# ag_valid = get_table_set(pd.read_csv(root_git+'balanced_datasets/valid.csv'))
ag_train = get_table_set(train)
ag_test = get_table_set(test)
ag_valid = get_table_set(valid)

In [ ]:
check_no_repetitions(ag_train, ag_test)

In [ ]:
check_no_repetitions(ag_train, ag_valid)

In [ ]:
check_no_repetitions(ag_valid, ag_test)

# Wikitables

## Train

Base

In [2]:
train_df = pd.read_csv(root_wiki+'/lsh_candidates/train_stats_2000000.csv')

In [ ]:
train_df['total_time'].isna().sum()

In [ ]:
plot_data_distribution(root_wiki+'/lsh_candidates/train_stats_2000000.csv')

High similarity

In [12]:
train_nb = pd.read_csv(root_wiki+'/lsh_candidates/high_similarity/train_stats.csv')

In [13]:
train_nb['a%'] = train_nb['a%'].fillna(0)

In [ ]:
plot_data_distribution(train_nb, 'a%')

In [15]:
train_nb = drop_from_interval(train_nb, 0,0.1, 93_636)[0]
train_nb = drop_from_interval(train_nb, 0.1,0.2, 173_023)[0]
train_nb = drop_from_interval(train_nb, 0.2,0.3, 305_233)[0]
train_nb = drop_from_interval(train_nb, 0.3,0.4, 144_848)[0]
train_nb = drop_from_interval(train_nb, 0.4,0.5, 75_015)[0]
train_nb = drop_from_interval(train_nb, 0.5,0.6, 127_137)[0]
train_nb = drop_from_interval(train_nb, 0.6,0.7, 20_342)[0]
train_nb = drop_from_interval(train_nb, 0.7,0.8, 4_713)[0]
train_nb = drop_from_interval(train_nb, 0.8,0.9, 48_002)[0]
train_nb = drop_from_interval(train_nb, 0.9,1, 212_411)[0]

In [ ]:
plot_data_distribution(train_nb, 'a%')

In [ ]:
train_nb.shape

In [19]:
train_nb = train_nb.drop(columns=['id'])

In [ ]:
train_nb

In [21]:
train_nb.to_csv(root_wiki+'/train.csv', index=False)

## Test

Base

In [40]:
test_df = pd.read_csv(root_wiki+'/lsh_candidates/test_stats_750000.csv')

In [ ]:
plot_data_distribution(root_wiki+'/lsh_candidates/test_stats_750000.csv')

In [44]:
test_augm = test_df[test_df['a%'] >= 0.6]
test_augm = test_augm[test_augm['a%'] < 0.8]

In [ ]:
plot_data_distribution(test_augm, 'a%')

High similarity

In [48]:
test_nb = pd.read_csv(root_wiki+'/lsh_candidates/high_similarity/test_stats.csv')

In [ ]:
plot_data_distribution(test_nb, 'a%')

In [50]:
test_nb['a%'] = test_nb['a%'].fillna(0)
test_nb = drop_from_interval(test_nb, 0,0.1,10_953)[0]
test_nb = drop_from_interval(test_nb, 0.1,0.2,16_629)[0]
test_nb = drop_from_interval(test_nb, 0.2,0.3,30_013)[0]
test_nb = drop_from_interval(test_nb, 0.3,0.4,14_674)[0]
test_nb = drop_from_interval(test_nb, 0.4,0.5,6_923)[0]
test_nb = drop_from_interval(test_nb, 0.5,0.6,13_537)[0]
test_nb = drop_from_interval(test_nb, 0.6,0.7,1_753)[0]
test_nb = drop_from_interval(test_nb, 0.7,0.8,437)[0]
test_nb = drop_from_interval(test_nb, 0.8,0.9,5_208)[0]
test_nb = drop_from_interval(test_nb, 0.9,1,23_495)[0]

In [ ]:
plot_data_distribution(test_nb, 'a%')

In [52]:
test_nb = test_nb.drop(columns=['id'])

In [ ]:
test_nb

In [54]:
test_nb.to_csv(root_wiki+'/test.csv', index=False)

## Valid

Base

In [7]:
valid_df = pd.read_csv(root_wiki+'/lsh_candidates/valid_stats_750000.csv')

In [ ]:
plot_data_distribution(root_wiki+'/lsh_candidates/valid_stats_750000.csv')

High similarity

In [ ]:
plot_data_distribution(root_wiki+'/lsh_candidates/high_similarity/valid_stats.csv')

In [56]:
valid_nb = pd.read_csv(root_wiki+'/lsh_candidates/high_similarity/valid_stats.csv')

In [ ]:
plot_data_distribution(valid_nb, 'a%')

In [58]:
valid_nb['a%'] = valid_nb['a%'].fillna(0)
valid_nb = drop_from_interval(valid_nb, 0, 0.1, 8_687)[0]
valid_nb = drop_from_interval(valid_nb, 0.1, 0.2, 19_913)[0]
valid_nb = drop_from_interval(valid_nb, 0.2, 0.3, 36_107)[0]
valid_nb = drop_from_interval(valid_nb, 0.3, 0.4, 16_003)[0]
valid_nb = drop_from_interval(valid_nb, 0.4, 0.5, 8_005)[0]
valid_nb = drop_from_interval(valid_nb, 0.5, 0.6, 12_964)[0]
valid_nb = drop_from_interval(valid_nb, 0.6, 0.7, 793)[0]
valid_nb = drop_from_interval(valid_nb, 0.7, 0.8, 102)[0]
valid_nb = drop_from_interval(valid_nb, 0.8, 0.9, 4_942)[0]
valid_nb = drop_from_interval(valid_nb, 0.9, 1, 21_038)[0]

In [ ]:
plot_data_distribution(valid_nb, 'a%')

In [ ]:
valid_nb

In [61]:
valid_nb = valid_nb.drop(columns=['id'])

In [ ]:
valid_nb

In [63]:
valid_nb.to_csv(root_wiki+'/valid.csv', index=False)

# New GitTables

## Train

In [30]:
train_triples = pd.read_csv(root_git+'/train.csv')

In [ ]:
plot_data_distribution(train_triples)

In [27]:
train_triples = drop_from_interval(train_triples, 0, 0.1, 833)[0]
train_triples = drop_from_interval(train_triples, 0.1, 0.2, 62_676)[0]
train_triples = drop_from_interval(train_triples, 0.2, 0.3, 11_845)[0]
train_triples = drop_from_interval(train_triples, 0.3, 0.4, 9_826)[0]
train_triples = drop_from_interval(train_triples, 0.4, 0.5, 28_478)[0]
train_triples = drop_from_interval(train_triples, 0.5, 0.6, 341_304)[0]
train_triples = drop_from_interval(train_triples, 0.6, 0.7, 73_939)[0]
train_triples = drop_from_interval(train_triples, 0.7, 0.8, 66_192)[0]
train_triples = drop_from_interval(train_triples, 0.8, 0.9, 440)[0]
train_triples = drop_from_interval(train_triples, 0.9, 1, 497_191)[0]

In [ ]:
plot_data_distribution(train_triples)

In [29]:
# train_triples.to_csv(root_git+'/train.csv', index=False)

## Test

In [41]:
test_triples = pd.read_csv(root_git+'/test.csv')

In [ ]:
plot_data_distribution(test_triples)

In [38]:
test_triples = drop_from_interval(test_triples, 0, 0.1, 13_795)[0]
test_triples = drop_from_interval(test_triples, 0.1, 0.2, 39_623)[0]
test_triples = drop_from_interval(test_triples, 0.2, 0.3, 9_786)[0]
test_triples = drop_from_interval(test_triples, 0.3, 0.4, 4_250)[0]
test_triples = drop_from_interval(test_triples, 0.4, 0.5, 16_937)[0]
test_triples = drop_from_interval(test_triples, 0.5, 0.6, 219_120)[0]
test_triples = drop_from_interval(test_triples, 0.6, 0.7, 33_127)[0]
test_triples = drop_from_interval(test_triples, 0.7, 0.8, 25_157)[0]
test_triples = drop_from_interval(test_triples, 0.8, 0.9, 509)[0]
test_triples = drop_from_interval(test_triples, 0.9, 1, 166547)[0]

In [ ]:
plot_data_distribution(test_triples)

In [40]:
# test_triples.to_csv(root_git+'/test.csv', index=False)

## Valid

In [ ]:
valid_triples = pd.read_csv(root_git+'/valid.csv')
plot_data_distribution(valid_triples)

In [44]:
valid_triples = drop_from_interval(valid_triples, 0, 0.1, 4_134)[0]
valid_triples = drop_from_interval(valid_triples, 0.1, 0.2, 23_875)[0]
valid_triples = drop_from_interval(valid_triples, 0.2, 0.3, 5_013)[0]
valid_triples = drop_from_interval(valid_triples, 0.3, 0.4, 1_960)[0]
valid_triples = drop_from_interval(valid_triples, 0.4, 0.5, 12_112)[0]
valid_triples = drop_from_interval(valid_triples, 0.5, 0.6, 123_963)[0]
valid_triples = drop_from_interval(valid_triples, 0.6, 0.7, 19_992)[0]
valid_triples = drop_from_interval(valid_triples, 0.7, 0.8, 16_206)[0]
valid_triples = drop_from_interval(valid_triples, 0.8, 0.9, 198)[0]
valid_triples = drop_from_interval(valid_triples, 0.9, 1, 148_751)[0]

In [ ]:
plot_data_distribution(valid_triples)

In [46]:
# valid_triples.to_csv(root_git+'/valid.csv', index=False)

# Table stats

In [ ]:
import pandas as pd
stats_git = pd.read_csv(root_git+'/table_stats/stats.csv')
print(stats_git.shape)
stats_git.describe()

In [ ]:

stats_wiki = pd.read_csv(root_wiki+'/table_stats/stats.csv')
print(stats_wiki.shape)
stats_wiki.describe()

In [1]:
import pandas as pd

In [ ]:
stats_wiki = pd.read_csv(root_wiki+'/table_stats/stats.csv')
stats_wiki

In [ ]:
stats_wiki.drop(index=[0,2])

# stuff

In [7]:
import pickle

In [8]:
with open(root_wiki+'/dictionaries/embedding_dictionaries/t_execs_armadillo_w_w.pkl','rb') as f:
    dd = pickle.load(f)

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv(root_wiki+'/test.csv')

In [ ]:
from tqdm import tqdm
tables = set(dd.keys())
missing = []
for r in tqdm(range(df.shape[0])):
    r_id = df.iloc[r].loc['r_id']
    s_id = df.iloc[r].loc['s_id']
    if r_id not in tables:
        missing.append(r_id)
    if s_id not in tables:
        missing.append(s_id)

In [ ]:
missing

In [14]:
import pandas as pd

In [ ]:
df = pd.read_csv(root_git+'/table_querying/evaluation/table_querying_results.csv') 
df.iloc[:,13:]

In [ ]:
df['bert_r_overlap_pred']

In [19]:
import pandas as pd

In [26]:
dd = pd.read_csv(root_git+'/table_querying/evaluation/table_querying_results.csv')

In [ ]:
dd['armadillo_g_g_ae'].mean()

In [ ]:
dd

In [32]:
def get_value_list(df):
    out = []
    for r in range(df.shape[0]):
        for c in range(df.shape[1]):
            out.append(df.iloc[r].iloc[c])
    return out

In [43]:
def get_value_list(df):
    df.values
    out = []

    return out

In [ ]:
get_value_list(dd)

In [ ]:
dd.iloc[:50, :10].shape

In [49]:
tmp_d = dd.iloc[:3,:3]

In [ ]:
get_value_list(tmp_d)

In [59]:
import itertools
def get_value_list(df):
    values = df.values.tolist()
    out = list(itertools.chain.from_iterable(values))
    return out

In [ ]:
len(get_value_list(dd))

In [ ]:
tmp_d

In [ ]:
tmp_d.values.tolist()

In [53]:
out = []
for l in tmp_d.values.tolist():
    out+=l

In [ ]:
out